In [10]:
import pandas as pd
import numpy as np
import torch.nn as nn
from guizero import *
import PIL
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True #imagenes ligeramente corruptas OK
from transformers import AutoImageProcessor
from transformers import AutoModelForImageClassification

In [11]:
#datos

In [12]:
data=None
try:
    data=pd.read_feather("Reclassified.feather")
except:
    data=pd.read_feather("Reclassified.bak.feather")

,Path,y,ny,my
0,images/2000/abr/p0423001.jpg,INA,INA,INA
1,images/2000/abr/p0423002.jpg,FUM,FUM,FUM
2,images/2000/abr/p0423003.jpg,FUM,FUM,FUM
3,images/2000/abr/p0424001.jpg,FUM,FUM,FUM
4,images/2000/abr/p0424002.jpg,FUM,FUM,FUM
...,...,...,...,...
21926,images/2023/mar/p0316234.jpeg,FUM,FUM,FUM
21927,images/2023/mar/p0317231.jpeg,EXP+FUM,EXP+FUM,EXP+FUM
21928,images/2023/mar/p0317232.jpeg,EXP+FUM,EXP+FUM,EXP+FUM
21929,images/2023/mar/p0317233.jpeg,EXP+FUM,EXP+FUM,EXP+FUM


In [57]:
data.describe()

,Path,y,ny,my
count,21931,21931,21931,21931
unique,20927,5,5,5
top,images/2008/may/p0502081.jpg,FUM,FUM,FUM
freq,32,8887,8976,8824


In [13]:
data[data["y"]!=data["my"]].head(10)

,Path,y,ny,my
73,images/2000/may/p0526001.jpg,FUM,INA,INA
95,images/2000/jun/p0610001.jpg,INA,INA,FUM
430,images/2000/nov/p1130001.jpg,FUM,INA,INA
626,images/2001/feb/p0206001.jpg,FUM,EXP+FUM,EXP+FUM
721,images/2001/abr/p0408011.jpg,FUM,,INA
908,images/2002/ene/p0130022.jpg,FUM,,INA
916,images/2002/feb/p0215021.jpg,FUM,,INA
1284,images/2003/feb/p0203031.jpg,EXP+FUM,,EXP
1332,images/2003/feb/p0222036.jpg,EXP+FUM,,FUM
1555,images/2003/ago/p0829031.jpg,FUM,,INA


In [14]:
if False: #setup from NN_INPUT with this, ignore for new data
    data["Path"]="images/"+data["Path"]
    data["y"]=np.full(data.shape[0],"UNK")
    data["y"]=np.where((data["FUM"]>0) & (data["EXP"]>0) & (data["y"]=="UNK"),"EXP+FUM",data["y"])
    data["y"]=np.where((data["FUM"]>0) & (data["y"]=="UNK"),"FUM",data["y"])
    data["y"]=np.where((data["EXP"]>0) & (data["y"]=="UNK"),"EXP",data["y"])
    data["y"]=np.where((data["INAC"]>0) & (data["y"]=="UNK"),"INA",data["y"])
    data.drop(columns=["FUM","EXP","INAC"],inplace=True)
    data["ny"]=np.full(data.shape[0],"")
    data["my"]=np.full(data.shape[0],"")
    data

In [15]:
# utils

In [16]:
def load_image(path):
    return PIL.Image.open(path)

In [17]:
def save(backup):
    data.to_feather(f"Reclassified{'.bak' if backup else ''}.feather")

In [60]:
save(True) #ensure backup

In [19]:
def get_next(col="ny"):
    f=data[data[col]==""]
    if(f.shape[0]==0):
        return None
    return f.iloc[0]

In [20]:
def set_label(idx,label,col="ny"):
    data.iloc[idx][col]=label

In [21]:
# classifier

In [22]:
model_name="MimModels/swinv2-base-patch4-window16-256-popo"

In [28]:
device="cuda"

In [29]:
model = AutoModelForImageClassification.from_pretrained(model_name).to(device)

In [27]:
image_processor = AutoImageProcessor.from_pretrained(model_name)

In [30]:
def process(image):
    return image_processor(images=image, return_tensors="pt").to(model.device)["pixel_values"]

In [31]:
def consume(pixels):
    logits=model(pixel_values=pixels).logits
    return nn.functional.softmax(logits, dim=-1).detach().cpu().numpy().flatten()

In [32]:
#p=consume(process(load_image(data.iloc[0]["Path"])))
#p

In [33]:
def classify(probabilities):
    idx=probabilities.argmax(-1).item()
    return model.config.id2label[idx]

In [34]:
#model.config.id2label

In [35]:
#model.config.id2label[4],data.iloc[0]["y"]

In [36]:
while(True):
    r_count=data[data["my"]==""].shape[0]
    print(f"Remaining: {r_count} ({r_count/data.shape[0]*100:.02f}%)",end="\r")
    nxt=get_next(col="my")
    if nxt is None:
        break
    try:
        probs=consume(process(load_image(nxt["Path"])))
        cl=classify(probs)
    except:
        cl="UNK"
    save(True)
    data.loc[nxt.name,"my"]=cl
    save(False)
    

In [37]:
#data["ny"]=np.where(data["y"]!=data["my"],"",data["y"])

In [38]:
#save(True)

In [39]:
#save(False)

In [40]:
# initial state

In [44]:
r_count=data[data["ny"]==""].shape[0]
m_count=r_count
nxt=get_next()
if nxt is None:
    raise "You're done, go home!"

In [45]:
# interface

In [53]:
app = App(bg="black")
app.set_full_screen()
w,h=app.width,app.height
r_count=data[data["ny"]==""].shape[0]
current_img=load_image(nxt["Path"])
scaled_w=current_img.size[0]/current_img.size[1]*h*.6
pic=Picture(app, image=current_img, width=int(scaled_w), height=int(h*.6))
progress=Text(app, text=f"Remaining: {r_count} ({100-r_count/m_count*100:.02f}% complete)", width="fill", height=1,  )
progress.text_size*=3
progress.text_color="white"
info=Text(app, text=f"Model says: {nxt['my']} vs. Dataset: {nxt['y']}", width="fill", height=1,  )
info.text_size*=3
info.text_color="white"
bottom=Box(app, layout="grid", width="fill", height="fill")
buttons=None

def buttonPressed(i):
    global r_count, nxt, current_img
    print(model.config.id2label[i],"pressed")
    save(True)
    data.loc[nxt.name,"ny"]=model.config.id2label[i]
    save(False)

    r_count=data[data["ny"]==""].shape[0]
    nxt=get_next(col="ny")
    progress.value=f"Remaining: {r_count} ({100-r_count/m_count*100:.02f}% complete)"
    info.value=f"Model says: {nxt['my']} vs. Dataset: {nxt['y']}"
    current_img=load_image(nxt["Path"])
    pic.image=current_img
    scaled_w=current_img.size[0]/current_img.size[1]*h*.7
    pic.resize(int(scaled_w), int(h*.8))
    
def makeButton(i):
    label=model.config.id2label[i]
    b=PushButton(bottom, text=label, grid=[i,0], padx=0, pady=0, width=w//int(5*20), height=2, command=buttonPressed, args=[i])
    b.bg="white"
    b.text_size*=3
    return b
buttons=[makeButton(i) for i in range(5)]

app.focus()
app.display()

FUM pressed
FUM pressed
EXP pressed
EXP+FUM pressed
INA pressed
INA pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
UNK pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
UNK pressed
EXP+FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
EXP+FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
UNK pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
FUM pressed
UNK pressed
FUM pressed
FUM pressed
FUM pressed
INA pres

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\alexi\mambaforge\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\PushButton.py", line 206, in _command_callback
    self._command()
  File "C:\Users\alexi\mambaforge\lib\site-packages\guizero\utilities.py", line 364, in <lambda>
    return lambda: func_name(*args)
  File "C:\Users\alexi\AppData\Local\Temp\ipykernel_44812\1573614262.py", line 27, in buttonPressed
    info.value=f"Model says: {nxt['my']} vs. Dataset: {nxt['y']}"
TypeError: 'NoneType' object is not subscriptable
